# FAISS Indexes Simple Evaluation 

https://github.com/facebookresearch/faiss/wiki/Faiss-indexes#indexlsh-and-its-relationship-with-cell-probe-methods

In [28]:
import faiss                   # make faiss available
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

k = 4                          # we want to see 4 nearest neighbors

In [29]:
xb[0]

array([0.19151945, 0.62210876, 0.43772775, 0.7853586 , 0.77997583,
       0.2725926 , 0.27646425, 0.8018722 , 0.95813936, 0.87593263,
       0.35781726, 0.5009951 , 0.6834629 , 0.71270204, 0.37025076,
       0.5611962 , 0.50308317, 0.01376845, 0.7728266 , 0.8826412 ,
       0.364886  , 0.6153962 , 0.07538124, 0.368824  , 0.9331401 ,
       0.65137815, 0.39720258, 0.78873014, 0.31683612, 0.56809866,
       0.8691274 , 0.4361734 , 0.8021476 , 0.14376682, 0.70426095,
       0.7045813 , 0.21879211, 0.92486763, 0.44214076, 0.90931594,
       0.05980922, 0.18428709, 0.04735528, 0.6748809 , 0.59462476,
       0.5333102 , 0.04332406, 0.5614331 , 0.32966843, 0.5029668 ,
       0.11189432, 0.6071937 , 0.5659447 , 0.00676406, 0.6174417 ,
       0.9121229 , 0.7905241 , 0.99208146, 0.95880175, 0.7919641 ,
       0.28525096, 0.62491673, 0.4780938 , 0.19567518], dtype=float32)

## IndexFlatL2, Exact Search for L2

In [51]:
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(D[:5])  

True
100000
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751738 7.20763   7.2511625]
 [0.        6.3235645 6.684581  6.799946 ]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.527987  7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[6.815506  6.8894653 7.3956795 7.4290257]
 [6.6041107 6.679695  6.7209625 6.828682 ]
 [6.4703865 6.8578606 7.0043755 7.036564 ]
 [5.573681  6.407543  7.1395226 7.3555984]
 [5.409401  6.232216  6.4173393 6.5743637]]


## IndexHNSWFlat, Hierarchical Navigable Small World graph exploration

In [50]:
M = 10                         # number of neighbors used in the graph. A larger M is more accurate but uses more memory
index = faiss.IndexHNSWFlat(d, M)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(D[:5])                  

True
100000
[[  0 393 363  78]
 [  1 555 364 617]
 [  2 304 101  13]
 [  3  18 182  64]
 [  4 288 178 381]]
[[0.        7.1751738 7.20763   7.2511625]
 [0.        6.3235645 6.799946  6.884479 ]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.527987  7.6628466 7.7909145]
 [0.        6.7638035 7.3900466 7.46482  ]]
[[381 477 588 329]
 [526 911 142  64]
 [838 527 425 637]
 [196 184 164 281]
 [526 377 917 161]]
[[6.8155107 7.429021  7.529812  7.660471 ]
 [6.604111  6.6797004 6.7209654 7.0770564]
 [6.470383  6.8578625 7.036566  7.2935686]
 [5.5736856 6.4075484 7.139516  7.713971 ]
 [5.409403  6.2322154 6.700432  6.730236 ]]


## IndexLSH, Locality-Sensitive Hashing (binary flat index)

In [52]:
n_bits = 2 * d  
index = faiss.IndexLSH(d, n_bits)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(D[:5])  

True
100000
[[  0   4 392  26]
 [  1 392 798 808]
 [  2 304 918 845]
 [  3 925 768 653]
 [  4 106 155   0]]
[[ 0. 19. 20. 20.]
 [ 0. 13. 14. 18.]
 [ 0. 15. 16. 17.]
 [ 0. 14. 18. 19.]
 [ 0. 16. 19. 19.]]
[[1787 1947  790 1597]
 [ 576  861 1249  923]
 [ 954 1354 1685 1143]
 [ 740  753  196   55]
 [1536 1174 1534  315]]
[[16. 17. 18. 20.]
 [13. 15. 15. 16.]
 [12. 14. 14. 14.]
 [17. 18. 18. 19.]
 [14. 16. 16. 17.]]


binary input

In [53]:
x_query = np.random.randint(2, size=(nq,d)).astype('float32')
x_train = np.random.randint(2, size=(nb,d)).astype('float32')
x_train[:2]

array([[0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
        1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
        0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0.,
        1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [63]:
n_bits = 2 * d  #The number of bits n_bits must be equal to 8, 12 or 16. The dimension d should be a multiple of m
lsh = faiss.IndexLSH (d, n_bits)
#lsh.train(x_train)
lsh.add (x_train)
D, I = lsh.search(x_train[:5], k) # sanity check
print(I)
print(D)
D, I = lsh.search (x_query, k)
print(I[:5])                   # neighbors of the 5 first queries
print(D[:5])  

[[    0 13084 36234  9767]
 [    1 24660 98795 17454]
 [    2 51110 48727 33680]
 [    3 24183 47399 22491]
 [    4 89813 79742 30319]]
[[ 0. 21. 23. 24.]
 [ 0. 18. 20. 21.]
 [ 0. 20. 22. 22.]
 [ 0. 19. 22. 22.]
 [ 0. 17. 19. 19.]]
[[28196 21419 31744  6680]
 [60172 38495 51069 45648]
 [60352 57919 46759 22293]
 [44845 60311 82097 20406]
 [49435 89045 99267 86838]]
[[19. 20. 22. 23.]
 [22. 22. 23. 23.]
 [18. 21. 21. 22.]
 [20. 21. 22. 23.]
 [19. 19. 19. 19.]]


In [74]:
from scipy.spatial import distance
print(distance.hamming([0, 1, 0], [0, 1, 0]) * 3)
print(distance.hamming([1, 0, 0], [0, 1, 0]) * 3)
print(distance.hamming(x_train[0], x_train[0]) * d )
print(distance.hamming(x_train[0], x_train[13084]) * d )

0.0
2.0
0.0
22.0
